In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
# matplotlib 한글화
import matplotlib as mpl
import matplotlib.font_manager as fm

mpl.rcParams['axes.unicode_minus'] = False

path = 'C:/Windows/Fonts/malgun.ttf'
font_name = fm.FontProperties(fname=path, size=50).get_name()
plt.rc('font', family=font_name)

In [16]:
# 감성사전 import

In [4]:
df = pd.read_excel('./감성사전_편집.xlsx')
df = df.rename(columns={'Unnamed: 0':'단어'})
df.set_index('단어', drop=True, inplace=True)
df

,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
단어,,,,,,,,,,,
가련하다,1,0,0,0,0,0,0,0,0,0,0
가슴 아프다,1,0,0,0,0,0,0,0,0,0,0
가슴앓이,1,0,0,0,0,0,0,0,0,0,0
가엾다,1,0,0,0,0,0,0,0,0,0,0
각박하다,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
광대,1,0,0,0,1,0,0,0,0,0,0
한계였다,1,0,0,0,0,0,0,0,0,0,0
멸망했다,1,0,0,0,0,0,0,0,0,0,0


In [5]:
from selenium import webdriver
url = input('url을 입력하세요\n')

if 'https://novel.naver.com/webnovel/detail?' in url :
    print("웹소설 분석을 시작합니다.")
        
elif 'https://novel.naver.com/webnovel/detail?' not in url: 
    print("네이버 웹툰 웹소설의 url을 넣어주세요.")


url을 입력하세요
https://novel.naver.com/webnovel/detail?novelId=1002530&volumeNo=102
웹소설 분석을 시작합니다.


In [6]:
res = requests.get(url)
driver = webdriver.Chrome()
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
path = '#content > div.section_area_viewer > div.viewer_container.dotum > div'
novel = soup.select(path)
text = novel[0].text

In [7]:
# 한국어만 출력하는 함수

In [8]:
def text_clean(doc):
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ ]', '', doc)
    return text

In [9]:
text = text_clean(text)
# text

In [10]:
from konlpy.tag import Okt
okt = Okt()

In [11]:
okt_li = okt.morphs(text)
word_li = [i for i in okt_li if len(i) >= 2]
# std_df = pd.read_csv('./stopwords-ko.txt')
# word_li = [i for i in word_li if i not in std_df ]

In [12]:
mood_count = {'슬픔': 0,'분노': 0,'기쁨': 0,'긴장감': 0,'무서운': 0,'증오': 0,\
              '차분히': 0,'섹시': 0,'브릿락': 0,'설렘': 0,'몽환': 0}
mood_words = {}

for word in word_li:
    if word in df.index:
        word_mood = df.columns[df.loc[word, :] == 1].values[0]
        if word_mood in mood_words:
            mood_count[word_mood] += 1
            mood_words[word_mood].append(word)
        else:
            mood_count[word_mood] += 1
            mood_words[word_mood] =[word]
for mood in mood_words.keys():
    mood_words[mood] = set(mood_words[mood])

In [13]:
mood_count

{'슬픔': 20,
 '분노': 18,
 '기쁨': 22,
 '긴장감': 16,
 '무서운': 13,
 '증오': 28,
 '차분히': 2,
 '섹시': 6,
 '브릿락': 3,
 '설렘': 3,
 '몽환': 20}

In [26]:
['슬픔','분노','기쁨','긴장감','무서운','증오','차분히','섹시','브릿락','설렘','몽환']

['슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', '차분히', '섹시', '브릿락', '설렘', '몽환']

In [14]:
counter={}
for i in word_li:
    for j in df.index:
        if i == j:
            if i in counter:
                counter[i] += 1
            else:
                counter[i] = 1

In [15]:
counter

{'부정': 1,
 '망할': 1,
 '짜증': 1,
 '폭발': 1,
 '괴물': 3,
 '거북': 22,
 '바다': 3,
 '정신': 5,
 '죽인': 2,
 '무너지게': 1,
 '비장': 1,
 '공격': 5,
 '움찔': 3,
 '당황': 1,
 '이익': 1,
 '타격': 1,
 '위력': 1,
 '망설이는': 1,
 '용기': 1,
 '열정': 1,
 '놀라': 1,
 '미친': 2,
 '절망': 1,
 '재앙': 1,
 '덤벼': 1,
 '악착같이': 1,
 '싸우는': 1,
 '중요한': 2,
 '힘들다': 1,
 '파괴': 1,
 '바람': 1,
 '귀한': 1,
 '어렵다': 1,
 '빨리': 1,
 '공포': 5,
 '제일': 1,
 '해결': 1,
 '아픈': 1,
 '저주': 1,
 '의식': 3,
 '결심': 1,
 '심장': 2,
 '조심': 1,
 '버리고': 1,
 '탐욕': 1,
 '놀랄': 1,
 '감정': 4,
 '놀라서': 1,
 '급히': 1,
 '신경': 1,
 '포기': 1,
 '완성': 1,
 '혼란': 1,
 '피해': 1,
 '불구': 2,
 '거짓말': 1,
 '사라져': 1,
 '성공한': 1,
 '안도': 1,
 '한숨': 1,
 '딱딱한': 1,
 '허망': 1,
 '실패': 1,
 '빌어먹을': 1,
 '안색': 1,
 '없다': 1,
 '깨진': 1,
 '자랑': 1,
 '문제': 2,
 '맞아': 1,
 '가슴': 1,
 '승리': 1,
 '불행': 1,
 '한계': 1,
 '특별한': 1,
 '흔들리는': 1,
 '걱정': 1,
 '자는': 1,
 '놀랍게도': 1,
 '영웅': 1,
 '고집': 1,
 '간직': 1,
 '솜씨': 1,
 '말쑥한': 1,
 '독한': 1,
 '끔찍한': 1,
 '도움': 1,
 '정상': 2,
 '젊은': 2,
 '위험': 1,
 '빠진': 1,
 '황천': 1,
 '친절한': 1,
 '싫을': 1,
 '친한': 1,
 '싸가지': 1,
 '새끼': 1,
 '

In [27]:
# 웹소설의 벡터 값

In [16]:
mood_count.values()

dict_values([20, 18, 22, 16, 13, 28, 2, 6, 3, 3, 20])

In [17]:
mood_words

{'슬픔': {'감정',
  '걱정',
  '무너지게',
  '부정',
  '불구',
  '불행',
  '사라져',
  '실패',
  '안색',
  '어렵다',
  '없다',
  '절망',
  '포기',
  '피해',
  '한숨',
  '힘들다'},
 '분노': {'고집',
  '공격',
  '덤벼',
  '딱딱한',
  '망할',
  '빌어먹을',
  '싸가지',
  '싸우는',
  '싸움',
  '악착같이',
  '재앙',
  '죽인',
  '파괴'},
 '증오': {'거북', '끔찍한', '버리고', '새끼', '싫을', '짜증', '탐욕'},
 '긴장감': {'급히',
  '놀라',
  '놀라서',
  '놀랄',
  '놀랍게도',
  '당황',
  '문제',
  '움찔',
  '중요한',
  '폭발',
  '한계',
  '혼란'},
 '무서운': {'거짓말', '공포', '괴물', '깨진', '위험', '저주', '황천'},
 '몽환': {'바다', '바람', '비장', '빨리', '신경', '심장', '의식', '자는', '정신', '조심', '허망'},
 '기쁨': {'간직',
  '결심',
  '귀한',
  '도움',
  '말쑥한',
  '성공한',
  '솜씨',
  '승리',
  '열정',
  '열중',
  '영웅',
  '완성',
  '용기',
  '이익',
  '자랑',
  '젊은',
  '정상',
  '제일',
  '특별한',
  '해결'},
 '브릿락': {'맞아', '아픈', '타격'},
 '설렘': {'위력', '친절한', '친한'},
 '차분히': {'망설이는', '안도'},
 '섹시': {'가슴', '독한', '미친', '빠진', '흔들리는'}}

In [ ]:
# 단어별 몇 번이나 등장하는지 체크 

In [51]:
word_counts = {}
for i in word_li:
    if i in word_counts:
        word_counts[i] += 1
    else:
        word_counts[i] = 1

In [52]:
sorted(word_counts.items(), key = lambda item: item[1], reverse=True)

[('강준', 29),
 ('아내', 20),
 ('희는', 16),
 ('으로', 13),
 ('처럼', 13),
 ('영준', 11),
 ('희가', 10),
 ('주세', 9),
 ('있었다', 9),
 ('하고', 8),
 ('얼굴', 8),
 ('서강준', 8),
 ('하는', 8),
 ('신부', 8),
 ('자신', 7),
 ('에게', 7),
 ('아이', 7),
 ('에서', 7),
 ('오늘', 7),
 ('눈빛', 7),
 ('했다', 6),
 ('희를', 6),
 ('신가', 6),
 ('엄마', 6),
 ('그리고', 5),
 ('아니라', 5),
 ('까지', 5),
 ('아빠', 5),
 ('만큼', 5),
 ('며느리', 5),
 ('였다', 5),
 ('이제', 5),
 ('가족', 4),
 ('에겐', 4),
 ('리틀', 4),
 ('작은', 4),
 ('미소', 4),
 ('천사', 4),
 ('아들', 4),
 ('했지만', 4),
 ('사랑', 4),
 ('숙이', 4),
 ('이었다', 4),
 ('아기', 4),
 ('없었다', 4),
 ('지금', 4),
 ('한결같이', 4),
 ('동화책', 4),
 ('시선', 4),
 ('부부', 4),
 ('사람', 4),
 ('실장', 4),
 ('신랑', 4),
 ('결혼', 4),
 ('남자', 4),
 ('이자', 3),
 ('서예', 3),
 ('한신', 3),
 ('희의', 3),
 ('새까만', 3),
 ('이번', 3),
 ('어머니', 3),
 ('고집', 3),
 ('정말', 3),
 ('출산', 3),
 ('에도', 3),
 ('그럼에도', 3),
 ('않았다', 3),
 ('시간', 3),
 ('고개', 3),
 ('안았다', 3),
 ('가슴', 3),
 ('소파', 3),
 ('둘째', 3),
 ('있는', 3),
 ('보다', 3),
 ('같은', 3),
 ('손끝', 3),
 ('결국', 3),
 ('여자', 3),
 ('빤히', 3),
 ('벌

In [ ]:
# 로맨스 https://novel.naver.com/webnovel/detail?novelId=994958&volumeNo=2
# 판타지 https://novel.naver.com/webnovel/detail?novelId=1002530&volumeNo=102
# 무협 https://novel.naver.com/webnovel/detail?novelId=927909&volumeNo=114
# 미스터리 https://novel.naver.com/webnovel/detail?novelId=987046&volumeNo=84


#### 가중치 +5

In [91]:
anger = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
temps = []

import time
url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=490101618'
driver.get(url)
time.sleep(0.15)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
for i in range(0, 38) :
    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    temps.append((songs, singers))
anger['노래'] = temps

In [92]:
anger['분노'] = 5
anger

,노래,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
0,"(어떤X (What Type of X), 제시 (Jessi))",NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(Break, 빈지노 (Beenzino))",NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(가라고 (Gotta Go), 선미)",NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"(야 우냐 (U MAD), BOBBY)",NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"(WHY DO FUCKBOIS HANG OUT ON THE NET, 키드밀리 (Ki...",NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"(부럽지가 않어, 장기하)",NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"(GIDDY UP (Prod. GroovyRoom), 식케이 (Sik-K))",NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"(Boss Bitch, 김하온 (HAON))",NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"(BOSS, pH-1)",NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"(Psycho, Woodie Gochild)",NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
scary = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
temps = []

url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=467106196'
driver.get(url)
time.sleep(0.15)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
for i in range(0, 31) :
    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    temps.append((songs, singers))
scary['노래'] = temps

In [94]:
scary['무서운'] = 5
scary

,노래,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
0,"(어느 날 (Duet With 김윤아), 이적)",NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
1,"(Ovum, 레이니 썬)",NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
2,"(이상한 얘기 (Daytripper), 하키)",NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
3,"(New 욕, 자우림)",NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
4,"(우린 결국 그렇게 (Feat. Seori), 기리보이)",NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
5,"(버려진 것들, 스웨덴세탁소)",NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
6,"(길 잃은 강아지, 아이유)",NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
7,"(전 여자친구에게 (Feat. 선우정아), San E)",NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
8,"(행복합니다 (Feat. 김종완 Of Nell), 에픽하이 (EPIK HIGH))",NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
9,"(요단강, 네스티요나)",NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
britrock = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
temps = []

url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=508377011'
driver.get(url)
time.sleep(0.15)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
for i in range(0, 44) :
    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    temps.append((songs, singers))
britrock['노래'] = temps

In [ ]:
britrock['브릿락'] = 5
britrock

In [ ]:
hatred = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
temps = []

url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=508377011'
driver.get(url)
time.sleep(0.15)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
for i in range(0, 18) :
    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    temps.append((songs, singers))
hatred['노래'] = temps

In [ ]:
hatred['증오'] = 5
hatred

##### 50개 이상은 50개만 긁어옴 : 기쁨, 슬픔, 차분한, 긴장감, 섹시, 설렘, 몽환

In [96]:
sad = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
temps = []

url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=403014132'
driver.get(url)
time.sleep(0.15)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
for i in range(0, 50) :
    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    temps.append((songs, singers))
sad['노래'] = temps

In [97]:
sad['슬픔'] = 5
sad

,노래,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
0,"(사랑은 늘 도망가, 임영웅)",5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(코 끝이 찡하게, 차가을)",5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(혼자 사랑하고 미워해, 백예슬)",5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"(비 오는 날 듣기 좋은 노래 (Feat. Colde), 에픽하이 (EPIK HIGH))",5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"(Your Lights, 이수 (엠씨더맥스))",5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"(너와 나 우리 (Feat. ZELO of B.A.P), 이예준)",5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"(별이 쏟아지는 밤, 려욱 (RYEOWOOK))",5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"(헤픈 우연, 헤이즈 (Heize))",5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"(Falling, Harry Styles)",5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"(High, 5 Seconds of Summer)",5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
happy = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
temps = []

url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=431707234'
driver.get(url)
time.sleep(0.15)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
for i in range(0, 50) :
    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    temps.append((songs, singers))
happy['노래'] = temps

In [99]:
happy['기쁨'] = 5
happy

,노래,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
0,"(Savior, 김성규)",NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(Shooting Star, 남우현)",NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(MAISON, 드림캐쳐)",NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"(Darl+ing, 세븐틴)",NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"(LOVE DIVE, IVE (아이브))",NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"(End Of Story, 유아 (오마이걸))",NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"(LA LA POP!, 하성운)",NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"(MY BAG, (여자)아이들)",NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"(봄이 피어나, 비투비)",NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"(사랑인가 봐 (Acoustic Ver.), 김세정)",NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
calm = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
temps = []

url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=476757748'
driver.get(url)
time.sleep(0.15)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
for i in range(0, 50) :
    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    temps.append((songs, singers))
calm['노래'] = temps

In [101]:
calm['차분히'] = 5
calm

,노래,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
0,"(In the whale, 온유 (ONEW))",NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN
1,"(시계초침 (Feat. BRWN, Huiii), SUDI)",NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN
2,"(So Long (Feat. I'll), THAMA)",NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN
3,"(흔적기관 (痕跡器管), 김뮤지엄 (KIMMUSEUM))",NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN
4,"(친구 사이, JEMN)",NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN
5,"(PILLOW, DOKO (도코))",NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN
6,"(Love Phobia, Wynn (윈))",NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN
7,"(그걸로도 충분해요 (Feat. 수안), 온유 (ONEW))",NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN
8,"(Yesterday, San E)",NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN
9,"(MDA, 초아)",NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN


In [102]:
sexy = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
temps = []

url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=443836057'
driver.get(url)
time.sleep(0.15)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
for i in range(0, 50) :
    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    temps.append((songs, singers))
sexy['노래'] = temps

In [103]:
sexy['섹시'] = 5
sexy

,노래,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
0,"(야몽음인 (夜夢陰人), 이바다)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN
1,"(One More Night (Feat. VINXEN), 베이빌론 (Babylon))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN
2,"(Deep, 유성은)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN
3,"(소원이지 (Feat. 유병재), MINO (송민호))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN
4,"(닿은 순간 (Ooh La La La), EXO)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN
5,"(비 오는 거리 (Feat. 핫펠트), 베이빌론 (Babylon))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN
6,"(깜빡 Remix (Feat. Crush, Elo & Jinbo), GRAY (그레이))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN
7,"(Funny (PROD. e.one, SEION), 소유 (SOYOU))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN
8,"(곡선, 선미)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN
9,"(HERE I COME, 신화)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN


In [104]:
nerve = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
temps = []

url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=412763124'
driver.get(url)
time.sleep(0.15)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
for i in range(0, 50) :
    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    temps.append((songs, singers))
nerve['노래'] = temps

In [105]:
nerve['긴장감'] = 5
nerve

,노래,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
0,"(Alive, 캐스커)",NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(Save me, leeSA (리싸))",NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(소나기, 우효)",NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,((Bonus) What Are You Seeing? (Performance in ...,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"(Teardrop, Massive Attack)",NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"(안개, 보아 (BoA))",NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"(Nobody Knows, 김윤아)",NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"(꽃이 지다 (Feat. 조원선), 정재형)",NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"(목소리, 김윤아)",NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"(숨결, Mad Soul Child)",NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
dokidoki = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
temps = []

url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=427186714'
driver.get(url)
time.sleep(0.15)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
for i in range(0, 50) :
    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    temps.append((songs, singers))
dokidoki['노래'] = temps

In [107]:
dokidoki['설렘'] = 5
dokidoki

,노래,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
0,"(봄눈, 펜타곤)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
1,"(첫 데이트, 스윗소로우 (SWEET SORROW))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
2,"(벚꽃 엔딩, 버스커 버스커)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
3,"(봄 사랑 벚꽃 말고, HIGH4 (하이포))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
4,"(우연히 봄, 아이유)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
5,"(봄날, 벚꽃 그리고 너, 로꼬)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
6,"(봄인가 봐 (Spring Love), 유주 (YUJU))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
7,"(One More Time, One More Chance (초속5센티미터 OST 2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
8,"(봄바람 (Feat. 나얼), 에릭남 (Eric Nam))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
9,"(너는 나의 봄이다, 웬디 (WENDY))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN


In [108]:
fantasy = pd.DataFrame(columns = ['노래','슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', \
                                  '차분히', '섹시', '브릿락', '설렘', '몽환'])
driver = webdriver.Chrome() 
temps = []

url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=427968315'
driver.get(url)
time.sleep(0.15)
path = '#frm > div'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
for i in range(0, 50) :
    songs = soup.select('div.ellipsis.rank01 > span > a')[i].text
    singers = soup.select('div.ellipsis.rank02 > span > a')[i].text
    temps.append((songs, singers))
fantasy['노래'] = temps

In [109]:
fantasy['몽환'] = 5
fantasy

,노래,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
0,"(사실 난, 디어클라우드)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
1,"(청춘 (Feat. 김창완), 김필)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
2,"(클로즈, 못 (Mot))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
3,"(내가 사랑한 모든 것들은 나를 눈물짓게 할 테니까, 모브닝 (MOVNING))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
4,"(어느 새(디깅클럽서울 Ver.), 백예린 (Yerin Baek))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5,"(Away, Collective Arts (콜렉티브아츠))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
6,"(그런 밤, 김현창)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
7,"(행복해 보이는 사람처럼, 어반자카파)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
8,"(어른, 빌리어코스티)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
9,"(꽃 (Prod. 로코베리), 이섭)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [4]:
def novel_score_extract(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    text = soup.select('div.detail_view_content')[0].text
    text = ' '.join(re.compile('[ㄱ-ㅎ가-힣]+').findall(text))

    okt = Okt()
    okt_li = okt.morphs(text)
    globals()['word_li'] = [i for i in okt_li if len(i) >= 2]

In [ ]:
# 각 노래 별 감성 점수 

In [18]:
emotion = pd.read_csv('./노래별감성점수_최종.csv')
emotion = emotion.drop(['Unnamed: 0'], axis=1)
emotion

,노래,가수,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
0,사랑은 늘 도망가,임영웅,9,0,0,1,0,0,0,2,0,11,4
1,코 끝이 찡하게,차가을,8,0,1,0,0,0,0,0,1,4,1
2,혼자 사랑하고 미워해,백예슬,7,1,1,0,0,2,0,2,0,6,0
3,비 오는 날 듣기 좋은 노래 (Feat. Colde),에픽하이 (EPIK HIGH),19,2,5,2,0,3,1,1,0,5,0
4,Your Lights,이수 (엠씨더맥스),6,0,2,0,0,2,0,1,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,Faded,Alan Walker,0,0,0,0,0,0,0,0,0,0,5
477,혼자라는 느낌,이상은 (=Lee-tzsche),5,0,0,2,0,1,0,4,0,2,7
478,재와 연기의 노래,못 (Mot),4,0,2,0,0,0,0,0,4,0,7
479,Scenery,Ashmute (애쉬뮤트),10,0,0,0,0,0,0,0,0,0,5


In [24]:
a = [20, 18, 22, 16, 13, 28, 2, 6, 3, 3, 20]
b = emotion.iloc[i, :].values

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

In [42]:
def search_similarity(a):
    similarity_li = []
    for i in range(len(emotion)):
        b = emotion.iloc[i, :].values
        similarity_li.append(cosine_similarity(a, b))
    print(b)
    song_score = emotion.iloc[np.argmax(similarity_li), 3:].values
    title = emotion.iloc[np.argmax(similarity_li), 1]
    singer = emotion.iloc[np.argmax(similarity_li), 2]
    return f"{singer} - {title}", song_score
    print(cos_sim(a, b))

In [43]:
# url = 'https://novel.naver.com/webnovel/detail?novelId=1002530&volumeNo=102'
novel_score = novel_score_extract(url)
music, song_score = search_similarity(novel_score)

NameError: name 'novel_score_extract' is not defined

In [41]:
print(cosine_similarity(a, b))

NameError: name 'b' is not defined

In [39]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [40]:
print(cos_sim(a, b))

NameError: name 'b' is not defined

In [28]:
url = 'https://novel.naver.com/webnovel/detail?novelId=1002530&volumeNo=102'
novel_score = novel_score_extract(url)
music, song_score = search_similarity(novel_score)

NameError: name 'novel_score_extract' is not defined

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

In [76]:
a = [9, 3, 23, 3, 2, 2, 9, 27, 5, 32, 16]

In [77]:
# 코사인 유사도 : 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도

In [78]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

In [7]:
def search_similarity(a):
    similarity_li = []
    for i in range(len(emotion)):
        b = emotion.iloc[i, :].values
        similarity_li.append(cosine_similarity(a, b))
    print(np.argmax(similarity_li))
    title = emotion.노래
    singer = emotion.가수
    return f"{singer} - {title}"